## `met` (near surface meteorology) Variable Summaries

This notebook will orchestrate the monthly decadal mean summaries of the following variables from the `met` set of files:


All this notebook does is stage the files, set up a Dask client, and then iterate through all models, scenarios, decades, variables, and months to produce the summary GeoTIFFs.

In [1]:
import os
import xarray as xr
import rasterio as rio
import numpy as np
import dask
import dask.array as da
from dask.distributed import Client
from dask import delayed
from tqdm.auto import tqdm
from pathlib import Path

# local
from config import DATA_DIR, OUTPUT_DIR
from config import models, scenarios, variable_di, precision_di, months, unit_di, summary_di, mo_names
from compute_summaries import *

In [2]:
var_set = "met"

We know from the EDA that HadGEM2-ES_rcp45_BCSD_met_2005.nc and HadGEM2-ES_rcp85_BCSD_met_2005.nc are missing some data for December. Let's just skip the HadGEM2-ES model for now.

In [3]:
models

['ACCESS1-3',
 'CanESM2',
 'CCSM4',
 'CSIRO-Mk3-6-0',
 'GFDL-ESM2M',
 'HadGEM2-ES',
 'inmcm4',
 'MIROC5',
 'MPI-ESM-MR',
 'MRI-CGCM3']

In [4]:
models.remove("HadGEM2-ES")
models

['ACCESS1-3',
 'CanESM2',
 'CCSM4',
 'CSIRO-Mk3-6-0',
 'GFDL-ESM2M',
 'inmcm4',
 'MIROC5',
 'MPI-ESM-MR',
 'MRI-CGCM3']

In [5]:
paths = list(DATA_DIR.glob(f"*{var_set}*.nc"))
paths = [x for x in paths if "HadGEM2-ES" not in x.name.split("_")] 
assert len(paths) == 3000 - 300 # value learned from EDA notebook

In [6]:
process_group_di = {}
for model in models:
    process_group_di[model] = {}
    for scenario in scenarios:
        process_group_di[model][scenario] = []

for fp in paths:
    fp_model = fp.name.split("_")[0]
    fp_scenario = fp.name.split("_")[1]
    process_group_di[fp_model][fp_scenario].append(fp)

for model in models:
    for scenario in scenarios:
        assert len(process_group_di[model][scenario]) == 150 # value learned from EDA notebook

In [7]:
# create dask client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 251.72 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37335,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 251.72 GiB
Comm: tcp://127.0.0.1:35243,Total threads: 4
Dashboard: http://127.0.0.1:36620/status,Memory: 31.46 GiB
Nanny: tcp://127.0.0.1:33596,


In [8]:
%%time

for model in models:
    for scenario in scenarios:

        ncube = mfload_all_netcdf_data(process_group_di[model][scenario])
        projcube, wrf_raster_profile = project_datacube(ncube)

        # keeping this snippet for convenient testing - just process a single decade
        # for decade_start in tqdm(range(1950, 1960, 10), desc=f"Processing {model} {scenario} data..."):
        for decade_start in tqdm(range(1950, 2100, 10), desc=f"Processing {model} {scenario} data..."):
            decade_slice = slice_by_decade(projcube, decade_start)
            
            for climvar in tqdm(variable_di[var_set].keys(), desc=f"Processing {decade_start}s"):
                decadal_means_of_monthly_summaries = compute_monthly_summaries(decade_slice, var_set, climvar)
                
                for month in months:
                    month_array = array_from_monthly_summary(decadal_means_of_monthly_summaries, climvar, month)
                    output_filename = make_output_filename(climvar, model, scenario, month, decade_start)
                    write_raster_to_disk(output_filename, wrf_raster_profile, month_array)

        projcube.close()

Processing ACCESS1-3 rcp45 data...:   0%|          | 0/15 [00:00<?, ?it/s]

Processing 1950s:   0%|          | 0/3 [00:00<?, ?it/s]

Processing 1960s:   0%|          | 0/3 [00:00<?, ?it/s]

Processing 1970s:   0%|          | 0/3 [00:00<?, ?it/s]

Processing 1980s:   0%|          | 0/3 [00:00<?, ?it/s]

...

Processing 2060s:   0%|          | 0/3 [00:00<?, ?it/s]

Processing 2070s:   0%|          | 0/3 [00:00<?, ?it/s]

Processing 2080s:   0%|          | 0/3 [00:00<?, ?it/s]

Processing 2090s:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 40min 26s, sys: 2min 48s, total: 43min 14s
Wall time: 1h 3min 31s


In [ ]:
client.close()